<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import requests 

In [ ]:
# 爬取 一张图片， 并做持久化保存
import requests
url = 'https://ss1.bdstatic.com/70cFvXSh_Q1YnxGkpoWK1HF6hhy/it/u=2381789298,2193118133&fm=26&gp=0.jpg'
# 对于反爬机制--UA检测(对请求载体的检测)， 应对的反反爬策略为： 添加请求头信息 User-Agent.
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}
f = open('one.jpg','wb')
img_data = requests.get(url=url,headers=header).content
f.write(img_data)

In [ ]:
# 正则匹配 URL 中 指定的字符串，例： ID 或 一张图片的文件名
import re
import requests 
url = 'https://ss1.bdstatic.com/70cFvXSh_Q1YnxGkpoWK1HF6hhy/it/u=2381789298,2193118133&fm=26&gp=0.jpg'
shi = re.search('.*y/it/u=(.*?),.*',url)
print(shi.groups())

In [3]:
# 爬取 搜狗网首页 源码并做持久化保存
import requests 
url = 'https://www.sogou.com/'
res = requests.get(url=url)
pg_text = res.text
with open('sogou.html','w') as f:
    f.write(pg_text)
print('over')

In [ ]:
#需求：爬取搜狗指定词条搜索后的页面数据,做持久化保存
import requests
url = 'https://www.sogou.com/web'
# 用户输入指定条目
wd = input('enter a word:')
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}
#封装参数
param = {
    'query':wd
}
response = requests.get(url=url,params=param,headers=header)

page_text = response.content
fileName = wd+'.html'
with open(fileName,'wb') as fp:
    fp.write(page_text)
print('over')

In [8]:
# 爬取 百度翻译中  指定词条 后返回的数据
# https://fanyi.baidu.com/translate
# 分析得知， 翻译结果数据 为 动态加载，是 Ajax 请求，返回的为 JSON 字典数据
import requests 
url = "https://fanyi.baidu.com/sug"
main = input("sou:")
data = {
    "kw": main
}
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}
res = requests.post(url=url,data=data,headers=header)
# 确保此请求 返回的数据为 JSON 数据，才可使用 .json() 方法 接受，否则报错
pg_text = res.json()  
print(pg_text)

sou:dog
{'errno': 0, 'data': [{'k': 'dog', 'v': 'n. 狗; 犬; 公狗; 公狐; 公狼; 赛狗; 灵𤟥赛; v. (长期)困扰，折磨，纠缠; 跟踪;'}, {'k': 'dogs', 'v': 'n. 狗; 犬; 公狗; 公狐; 公狼; 赛狗; 灵𤟥赛; v. (长期)困扰，折磨，纠缠; 跟踪;'}, {'k': 'doge', 'v': 'n. （美西部）孤犊，无母犊牛;'}, {'k': 'doggy', 'v': 'n. (儿语)小狗，汪汪，狗狗; adj. 狗的; 像狗一样的;'}, {'k': 'doggie', 'v': 'n. (儿语)狗狗;'}]}


In [7]:
#  爬取  豆瓣电影  (按照热度)
# https://movie.douban.com/explore
# 分析得知: 页面电影信息为动态加载，
# 当滚轮触底或点击加载 更多时，发起 Ajax 请求，返回 JSON 字典电影数据
import requests 
url = 'https://movie.douban.com/j/search_subjects'
limit = int(input('数量:'))
start = int(input('开始位置:'))
# 构造 请求 URL 参数
data = {
    "type":"movie",
    "tag": "热门",
    "sort": "recommend",
    "page_limit": limit,
    "page_start": start,
}
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}
res = requests.get(url=url,params=data,headers=header)
# 接受 JSON 字典数据
pg_text = res.json()
# 做持久化保存
f = open('豆瓣Movie','w')
for s in pg_text['subjects']:
    entry = s['title'] + "\t" + s['rate'] + '\t' + s['url']
    print(entry)
    f.write(entry + '\r\n')
f.close()
print('over')

数量:2
开始位置:2
只有爱能让我生存	7.5	https://movie.douban.com/subject/30139756/
大人物	6.3	https://movie.douban.com/subject/30158840/


In [17]:
#需求：爬取国家药品监督管理总局中基于中华人民共和国化妆品生产许可证  每家公司的  相关数据
# http://125.35.6.84:81/xk/
# 第一步：************************ 分析：*****判断页面数据是否为动态加载
# 得知：
# 首页数据 为 动态加载，直接爬取首页 获取不到想要信息
# 得知 分页 url 为 Ajax 请求，只是 不同的页码 所带的参数不一样，返回的为 json 字典，
# 且 每一家企业的 详情页 的数据 也为 动态加载 ,为 Ajax 请求，返回的 为 json 字典
# 通过 分页 URL 返回到的 json数据，得到 每一家企业的 ID， 
# 得知 详情页的 URL 都一样，只是 不同的企业 所带的参数 ID 不一样。
# 再通过 分页 URL获得 ID 构造 详情页的 URL 
# 首页 分页的 URL 地址,
import requests 
pg_url = "http://125.35.6.84:81/xk/itownet/portalAction.do?method=getXkzsList"
busi_url = "http://125.35.6.84:81/xk/itownet/portalAction.do?method=getXkzsById"
start = int(input('起始页:'))
num = int(input("页数:"))
step = int(input('每页的数量:'))/2
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}

busi_list = []
f = open('company.html','w',encoding='utf-8')
for pag in range(start,start+num):     # 循环 爬取 每一页的 URL地址，
    pg_data = {
        "on": "true",
        "page": pag,
        "pageSize": step,
        "productName": "",
        "conditionType": "1",
        "applyname": "",
        "applysn": "",
    }
# 爬取 分页 URL， 获得 JSON 数据
    pg_list = requests.post(url=pg_url,data=pg_data,headers=header).json()
    for p in pg_list['list']:
        busi_data = {
            "id": p["ID"],
        }
# 爬取 详情页 URL, 获得 JSON 数据
        busi = requests.post(url=busi_url,data=busi_data,headers=header).json()
       
        busi_list.append(str(busi) + "\r\n")
# 将所有 企业的 信息 列表 数据 做持久化保存
f.writelines(busi_list)
f.close()
print('over')

起始页:2
页数:3
每页的数量:30


In [ ]:
## 百度搜索 --嗅图--: 爬取 照片
# 分析:
# 得知 页面数据 为动态加载，
# 当滚轮 触底 时 自动 触发 Ajax 请求，返回 JSON 数据。
import requests
import re
import os
pg_url = "https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord=嗅图&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=&z=&ic=&hd=&latest=&copyright=&word=嗅图&s=&se=&tab=&width=&height=&face=&istype=&qc=&nc=&fr=&expermode=&force=&pn=150&rn=100&gsm=96&1558065706969="
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
}
res = requests.get(url=pg_url,headers=header).json()
url_list = []
for s in res['data'][:60]:
    img_url = s['hoverURL']
    if img_url:
        url_list.append(img_url)
#         print(img_url)
if not os.path.exists('./xiutu'):
    os.mkdir('./xiutu')
for m in url_list:
    print('url=',m)
    file_name ='xiutu/'+ re.search(".*/it/u=(.*?),.*",m).groups()[0] + '.jpg'
    f = open(file_name,'wb')
    img_data = requests.get(url=m,headers=header).content
    f.write(img_data)
    f.close()
print('over')
